In [2]:
# early iteration, not used

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import zscore, rankdata
from heapq import nlargest
import random
import copy
import math

In [3]:
# head to head format
# use steamers projections for user
cwd = os.getcwd()
c_path = os.path.join(cwd, "Data", "Catchers 2020 Projections.csv")
first_path = os.path.join(cwd, "Data", "First Basemen 2020 Projections.csv")
second_path = os.path.join(cwd, "Data", "Second Basemen 2020 Projections.csv")
short_path = os.path.join(cwd, "Data", "Shortstop 2020 Projections.csv")
third_path = os.path.join(cwd, "Data", "Third Basemen 2020 Projections.csv")
outfield_path = os.path.join(cwd, "Data", "Outfielders 2020 Projections.csv")
dh_path = os.path.join(cwd, "Data", "Designated Hitters 2020 Projections.csv")
pitchers_path = os.path.join(cwd, "Data", "Pitchers 2020 Projections.csv")

# read csvs into dataframes
catchers_proj = pd.read_csv(c_path)
first_proj = pd.read_csv(first_path)
second_proj = pd.read_csv(second_path)
ss_proj = pd.read_csv(short_path)
third_proj = pd.read_csv(third_path)
of_proj = pd.read_csv(outfield_path)
dh_proj = pd.read_csv(dh_path)
pitchers_proj = pd.read_csv(pitchers_path)

# create a dictionary of positional databases
pos_players = {'C': catchers_proj, '1B': first_proj, '2B': second_proj, '3B': third_proj, 'SS': ss_proj\
               , 'OF': of_proj}

pos_cat = {'R':1, 'H':1, 'RBI':1, 'BB':1, 'SO':-1, 'SB':1}

# multiply the pitchers overall stats for each by their point totals, 
pitchers_cat = {'IP':3, 'H':-1, 'ER':-2, 'BB':-1, 'K':1, 'W':5, 'L':-5, 'SV':5}

# create a dictionary of the spots that each team is drafting
glob_spots = {'C': 1, '1B': 2, '2B': 2, '3B': 2, 'SS': 2, 'OF': 5, 'UTIL': 1}

In [4]:
# for an input position and dataframe, find the average value of each player
def avg_val_pos(pos, dataframe, categories):
    qual_player = dataframe
    qual_col = qual_player[list(categories.keys())]
    norm_col = qual_col.apply(zscore)
    qual_player['Pos'] = pos
    qual_player['Average Value'] = norm_col.mean(axis=1)
    return qual_player[['Name','Pos', 'Average Value']]

# create a dataframe of the combined positons with each player's average value
def combine_df(dict_dataset, game_min, categories):
    pos_val_df = pd.DataFrame(columns=['Name','Pos', 'Average Value'])
    for key, value in dict_dataset.items():
        value = value[value.G > game_min]
        value = value.reset_index()
        value.drop('index', axis=1, inplace=True)
        val = (avg_val_pos(key, value, categories))
        pos_val_df = pos_val_df.append(val, ignore_index=True)
    pos_val_df = pos_val_df.sort_values('Average Value', ascending = False)
    no_dups_table = pos_val_df.drop_duplicates(subset='Name', keep='first')
    no_dups_table = no_dups_table.reset_index()
    no_dups_table.drop('index', axis=1, inplace=True)
    return no_dups_table

combine_df(pos_players, 98, pos_cat)

,Name,Pos,Average Value
0,Mike Trout,OF,1.822045
1,Ronald Acuna Jr.,OF,1.719411
2,Cody Bellinger,1B,1.657424
3,Christian Yelich,OF,1.562399
4,Bryce Harper,OF,1.375250
...,...,...,...
214,Austin Nola,1B,-1.253760
215,Rio Ruiz,3B,-1.255440
216,Adam Haseley,OF,-1.288922
217,Lewis Brinson,OF,-1.295067


In [5]:
data = combine_df(pos_players, 100, pos_cat)
if 'Mike Trout' in data.Name.values:
    print(True)

True


In [6]:
# create a random draft order given the number of teams and number of spots for each position
def create_draft_order(num_teams, num_spots):
    draft_pos = list(range(1,num_teams+1))
    draft_order = random.sample(draft_pos, num_teams) # randomize the draft order 
    x = 1
    
    all_teams_spots = {}
    
    # copy the number of spots necessary for each team
    while x < 11:
        all_teams_spots['team{0}'.format(x)]= copy.deepcopy(num_spots)
        all_teams_spots['team{0}'.format(x)].update({'Draft Order' : draft_order[x - 1]})
        x += 1
    return draft_order, all_teams_spots

# pick the top player in the database and return their name and position
def pick_player(df, user=True, spots=None):
    if not user:
        for index, player in df.iterrows():
            if spots[player.Pos] > 0:
                return player.Name, player.Pos
    highest_player = df.iloc[0]
    return highest_player.Name, highest_player.Pos
        
# create_draft_order(10, spots)

def remove_pos(df, pos):
    return df[df.Pos != pos]

def remove_draftee(df, draftee):
    df = df[df.Name!=draftee]
    df = df.reset_index()
    df.drop('index', axis=1, inplace=True)
    return df

def sort_normed_df(norm_df):
    norm_df = norm_df.sort_values('Average Value', ascending=False)
    norm_df = norm_df.reset_index()
    norm_df.drop('index', axis=1, inplace=True)
    return norm_df

data = combine_df(pos_players, 80, pos_cat)
data = remove_pos(data, 'OF')
print (data)

                 Name Pos  Average Value
0     Yasmani Grandal   C       1.971312
2       J.T. Realmuto   C       1.888429
5      Cody Bellinger  1B       1.616607
7        Jose Ramirez  3B       1.414325
8        Trevor Story  SS       1.335255
..                ...  ..            ...
246       Austin Nola  1B      -1.105592
250      Miguel Rojas  SS      -1.258153
251   Tommy La Stella  3B      -1.276820
254     Johan Camargo  3B      -1.363645
255  Ryan Mountcastle  SS      -1.643133

[150 rows x 3 columns]


In [7]:
# given a drafted player, redistribute the user's database. if the user drafted the player, then need to update
# their team's spots and the database with a discount factor
def redist_df(dataframe, spots, player, discount, drafted=False):
    if dataframe.empty:
        return dataframe
    row_index = dataframe[dataframe.Name==player].index
    row = dataframe.iloc[row_index]
    dataframe.drop(row_index, inplace=True)
    drafted_pos = (row.loc[row_index, 'Pos'].values)[0]
    normed_df = dataframe[['Name', 'Pos']]
    normed_df['Average Value'] = dataframe[['Average Value']].apply(zscore)
    if drafted:
        if drafted_pos in spots:
            if spots[drafted_pos] > 0:
                data = spots[drafted_pos]
#                 discount_factor = 1 - (1 / data)
                discount_factor = discount
            elif spots['UTIL'] > 0:
                spots[drafted_pos] += 1
#                 discount_factor = 1 - (1 / spots['UTIL'])
                discount_factor = 1
                spots['UTIL'] -= 1
            else:
                normed_df = remove_pos(normed_df, drafted_pos)
                discount_factor = 0
        else:
            return 'Unknown position'
        for index, row in normed_df.iterrows():
            if normed_df.loc[index, 'Pos'] == drafted_pos:
                if discount_factor == 0:
                    normed_df.loc[index, 'Average Value'] = 0
                elif normed_df.loc[index, 'Average Value'] >= 0:
                    normed_df.loc[index, 'Average Value'] = normed_df.loc[index, 'Average Value'] * discount_factor
                else:
                    normed_df.loc[index, 'Average Value'] = normed_df.loc[index, 'Average Value'] * (1 / discount_factor)
        return sort_normed_df(normed_df)
    return sort_normed_df(normed_df)

my_dataset = combine_df(pos_players, 100, pos_cat)
cell_spots = glob_spots
first_pick = redist_df(my_dataset, cell_spots, 'Mike Trout', 0.9, True)
print(first_pick)
second_pick = redist_df(first_pick, cell_spots, 'Cody Bellinger', 0.8, True)
print(second_pick)
third_pick = redist_df(second_pick, cell_spots, 'Yasmani Grandal', 0.7, True)
print(third_pick)
fourth_pick = redist_df(third_pick, cell_spots, 'Miguel Rojas', 0.9, True)
print(fourth_pick)
fifth_pick = redist_df(fourth_pick, cell_spots, 'Matt Carpenter', 0.8)
print(fifth_pick)
print(cell_spots)

                 Name Pos  Average Value
0      Cody Bellinger  1B       2.311808
1    Ronald Acuna Jr.  OF       2.160664
2    Christian Yelich  OF       1.965962
3        Trevor Story  SS       1.880663
4        Jose Ramirez  3B       1.879435
..                ...  ..            ...
201       Colin Moran  3B      -1.799669
202    Ender Inciarte  OF      -1.904258
203    Alex Dickerson  OF      -1.920246
204      Miguel Rojas  SS      -2.019193
205      Adam Haseley  OF      -2.206919

[206 rows x 3 columns]
                 Name Pos  Average Value
0    Ronald Acuna Jr.  OF       2.220586
1    Christian Yelich  OF       2.024419
2        Trevor Story  SS       1.938479
3        Jose Ramirez  3B       1.937242
4        Bryce Harper  OF       1.760210
..                ...  ..            ...
200    Alex Dickerson  OF      -1.891014
201      Miguel Rojas  SS      -1.990705
202     Albert Pujols  1B      -2.015057
203      Adam Haseley  OF      -2.179843
204      Ryan O'Hearn  1B      -2

In [8]:
def round_selection(round_order, val_df, opp_df, teams, all_teams_spots, discount):
    no_more_picks = False
    for team_num in round_order:
        if opp_df.empty:
            no_more_picks = True
            return val_df, opp_df, teams, all_teams_spots, no_more_picks
        if team_num == 1:
            draftee, draftee_pos = pick_player(val_df)
            all_teams_spots['team{0}'.format(team_num)][draftee_pos] -= 1
            val_df = redist_df(val_df, all_teams_spots['team{0}'.format(team_num)], draftee, discount, True)
        else:
            draftee, draftee_pos = pick_player(opp_df, False, all_teams_spots['team{0}'.format(team_num)])
            all_teams_spots['team{0}'.format(team_num)][draftee_pos] -= 1
            if draftee in val_df.Name.values:
                val_df = redist_df(val_df, all_teams_spots['team{0}'.format(team_num)], draftee, discount)
        opp_df = remove_draftee(opp_df, draftee)
        teams[team_num].append(draftee)
    return val_df, opp_df, teams, all_teams_spots, no_more_picks

In [9]:
# def test(dict_of_datasets, num_teams, spots, opp_dataset=None):

def test(user_df, num_teams, spots, discount):
    
    rounds = sum(spots.values()) + 1
    
    teams = dict((el,[]) for el in range(1, num_teams+1))
    
    forward_order, all_teams_spots = create_draft_order(num_teams, spots)
    
    first_drafter = forward_order.index(1) + 1
    
    opp_df = copy.copy(user_df)
        
    # in order to keep track of who is drafting, need to substract a round each time the person with the 
    # first pick drafts, since it is snake draft and they will have the last pick of the draft
    
    reverse_order = [element for element in reversed(forward_order)]
    round_num = 1
    no_more_picks = user_df.empty
    while round_num < rounds and not no_more_picks:
        if round_num % 2 == 1:
            round_order = forward_order
            user_df, opp_df, teams, all_teams_spots, no_more_picks = round_selection(round_order, user_df, opp_df, teams, all_teams_spots, discount)
        else:
            round_order = reverse_order
            user_df, opp_df, teams, all_teams_spots, no_more_picks = round_selection(round_order, user_df, opp_df, teams, all_teams_spots, discount)   
        round_num += 1
    return teams
                
    
# test(pos_players, 10, glob_spots, opp_proj)
my_data = combine_df(pos_players, 98, pos_cat)
test(my_data, 10, glob_spots, 0.9)

{1: ['Yasmani Grandal',
  'Trea Turner',
  'Peter Alonso',
  'Jorge Soler',
  'Keston Hiura',
  'Carlos Santana',
  'Brian Anderson',
  'Ramon Laureano',
  'Carlos Correa',
  'Vladimir Guerrero Jr.',
  'Byron Buxton',
  'Eddie Rosario',
  'Jonathan Schoop',
  'Mark Canha'],
 2: ['Ronald Acuna Jr.',
  'Rafael Devers',
  'Adalberto Mondesi',
  'Ozzie Albies',
  'Marcus Semien',
  'Jose Abreu',
  'Jorge Polanco',
  'Adam Eaton',
  'Franmil Reyes',
  'Edwin Encarnacion',
  'Miguel Sano',
  'Anthony Santander',
  'Gregory Polanco',
  'Wilson Ramos'],
 3: ['Mike Trout',
  'Javier Baez',
  'Fernando Tatis Jr.',
  'Xander Bogaerts',
  'Whit Merrifield',
  'Josh Bell',
  'Marcell Ozuna',
  'Bryan Reynolds',
  'David Dahl',
  'Buster Posey',
  'Eloy Jimenez',
  'Kyle Seager',
  'Miguel Cabrera',
  'Justin Turner'],
 4: ['Jose Ramirez',
  'J.D. Martinez',
  'Alex Bregman',
  'Rhys Hoskins',
  'Starling Marte',
  'Shin-Soo Choo',
  'Charlie Blackmon',
  'Lorenzo Cain',
  'Jeff McNeil',
  'Matt Car

In [10]:
all_data = pd.concat(list(pos_players.values()))
print(all_data)

                   Name       Team    G   PA   AB    H  2B  3B  HR   R  ...  \
0       Yasmani Grandal  White Sox  128  550  458  110  21   2  26  74  ...   
1         J.T. Realmuto   Phillies  128  558  505  136  29   2  24  74  ...   
2          Buster Posey     Giants  117  516  457  126  27   1  11  58  ...   
3     Christian Vazquez    Red Sox  104  411  377   99  20   1  12  47  ...   
4          Gary Sanchez    Yankees  107  452  398   96  16   1  30  65  ...   
...                 ...        ...  ...  ...  ...  ...  ..  ..  ..  ..  ...   
1567      Adolis Garcia    Rangers   32  134  125   29   6   1   5  15  ...   
1568       Sam Hilliard    Rockies   78  310  281   70  12   3  12  34  ...   
1569    Franchy Cordero     Padres   96  373  338   76  14   3  11  35  ...   
1570       Raimel Tapia    Rockies   77  317  294   82  17   3   6  35  ...   
1571      Lewis Brinson    Marlins   99  395  359   78  16   3  10  35  ...   

      wRC+  BsR  Fld  -1.2   Off   Def  WAR  -1.3  

In [11]:
def match_category_score_to_team(team_list, data, categories):
    team_cats = dict((el,0) for el in categories.keys())
    for team_player in team_list:
        player_in_db = data[data.Name == team_player]
        for stat, value in categories.items():
            team_cats[stat] += player_in_db[stat].values[0] * value
    return team_cats

ex = ['Yasmani Grandal','Jose Ramirez','Anthony Rizzo','Ketel Marte','Josh Donaldson','Giancarlo Stanton',
      'Carlos Santana','Max Kepler','Andrew McCutchen','Jorge Polanco','Shin-Soo Choo','Khris Davis',
      'Jonathan Schoop','Didi Gregorius','Buster Posey']
categories = {'R':1, 'H':1, 'RBI':1, 'BB':1, 'SO':-1, 'SB':1}
match_category_score_to_team(ex, all_data, categories)

{'R': 1294, 'H': 2134, 'RBI': 1274, 'BB': 986, 'SO': -1785, 'SB': 93}

In [12]:
def all_teams_stats(teams_dict, data, categories):
    teams_statistics = dict((el,0) for el in teams_dict.keys())
    for team, players in teams_dict.items():
        team_stats = match_category_score_to_team(players, data, categories)
        teams_statistics[team] = team_stats
    return teams_statistics

my_data = combine_df(pos_players, 100, pos_cat)
my_test = test(my_data, 10, glob_spots)

TypeError: test() missing 1 required positional argument: 'discount'

In [ ]:
teams_stats = all_teams_stats(my_test, all_data, categories)
print(teams_stats)

In [13]:
def sep_by_cat(teams_stats_dict, categories):
    cats = dict((el,[]) for el in range(len(categories)))
    for team, stats in teams_stats_dict.items():
        x = 0
        for stat, num in stats.items():
            cats[x].append(num)
            x += 1
    return cats
    
def create_rankings(teams_stats_dict, cat_lists):
    rankings = []
    total = []
    team_sum = dict((el,[]) for el in range(len(teams_stats_dict.values())))
    for key, value in cat_lists.items():
        rankings.append(list(rankdata(value).astype(int)))
    
    for stats in rankings:
        for y in range(len(stats)):
            team_sum[y].append(stats[y])
    for ranks in team_sum.values():
        total.append(sum(ranks))
    return total

def rank_teams(teams_stats_dict, categories):
    cats = sep_by_cat(teams_stats_dict, categories)
    final_points = create_rankings(teams_stats_dict, cats)
    return final_points

rank_teams(teams_stats, categories)

NameError: name 'teams_stats' is not defined

In [14]:
my_data = combine_df(pos_players, 110, pos_cat)
my_test = test(my_data, 10, glob_spots)
teams_stats = all_teams_stats(my_test, all_data, categories)
rank_teams(teams_stats, categories)

TypeError: test() missing 1 required positional argument: 'discount'

In [15]:
from scipy.stats import rankdata

def absolute_ranking(num_teams, cat_points):
    for cat in cat_points:
        rev_ranks = rankdata(cat, 'max')
        ranks = [num_teams + 1 - el for el in rev_ranks]
        print(ranks)

In [47]:
import scipy.stats as st

# 98 games is the highest number of games where there are enough of each position for each team to draft
games_min = [50,60,70,80,90,95,98]
discounts = [0.5, 0.7, 0.8, 0.9, 0.95, 0.99, 1.0, 1.5]

def run(pos_players_dict, all_data, categories, spots, num_teams, games_min, trials, discount=0.8, confidence=0.95):
    average_points = []
    for x in range(trials):
        combined_df = combine_df(pos_players_dict, games_min, categories)
        draft = test(combined_df, num_teams, spots, discount)
        teams_stats = all_teams_stats(draft, all_data, categories)
        average_points.append(rank_teams(teams_stats, categories))
    mean, sem, m = np.mean(average_points, axis=0), st.sem(average_points), st.t.ppf((1+confidence)/2., trials-1)
    lower, upper = mean - m*sem, mean + m*sem
    rounded_mean, rounded_lower, rounded_upper = [round(num, 1) for num in mean], [round(num, 1) for num in lower], [round(num, 1) for num in upper]
    return [rounded_mean, rounded_lower, rounded_upper]
            
        
run(pos_players, all_data, categories, glob_spots, 10, 100, 10, 0.9, 0.95)

[[28.7, 32.6, 31.6, 33.7, 38.1, 30.7, 34.0, 31.0, 34.4, 33.5],
 [24.4, 27.2, 26.6, 26.6, 32.7, 23.3, 28.8, 25.0, 28.5, 27.8],
 [33.0, 38.0, 36.6, 40.8, 43.5, 38.1, 39.2, 37.0, 40.3, 39.2]]

In [48]:
num_teams = 10
trials = 30
confidence = 0.95

for discount in discounts:
    for num_games in games_min:
        trial_run = run(pos_players, all_data, categories, glob_spots, num_teams, num_games, trials, discount, confidence)
        print(trial_run)
        print('Discount factor: ' + str(discount) + ', number of games: ' + str(num_games))

[[30.3, 33.0, 33.8, 32.6, 35.2, 30.7, 30.6, 33.1, 35.2, 33.5], [28.9, 30.2, 31.4, 30.0, 32.0, 27.6, 27.9, 30.3, 33.2, 31.0], [31.7, 35.8, 36.2, 35.3, 38.3, 33.8, 33.3, 35.9, 37.2, 36.1]]
Discount factor: 0.5, number of games: 50
[[30.2, 33.3, 33.8, 35.9, 32.6, 33.3, 34.8, 32.3, 30.7, 31.4], [28.0, 30.2, 31.4, 33.6, 29.7, 30.5, 32.0, 29.6, 27.7, 28.9], [32.3, 36.5, 36.2, 38.2, 35.5, 36.1, 37.5, 35.0, 33.6, 33.9]]
Discount factor: 0.5, number of games: 60
[[27.1, 30.6, 35.0, 32.9, 32.8, 34.0, 35.2, 32.2, 34.1, 33.5], [24.7, 28.0, 32.4, 30.3, 29.8, 31.3, 32.8, 29.9, 31.6, 30.9], [29.5, 33.1, 37.5, 35.4, 35.8, 36.8, 37.5, 34.5, 36.6, 36.0]]
Discount factor: 0.5, number of games: 70
[[28.9, 34.2, 34.3, 34.1, 35.1, 32.8, 31.1, 31.3, 30.9, 35.8], [26.4, 31.4, 31.7, 31.8, 32.1, 29.8, 27.7, 28.4, 27.5, 33.5], [31.5, 37.0, 36.9, 36.4, 38.1, 35.8, 34.4, 34.2, 34.4, 38.2]]
Discount factor: 0.5, number of games: 80
[[26.8, 30.7, 33.2, 32.2, 36.9, 35.5, 33.1, 33.6, 33.1, 32.6], [24.8, 28.1, 31.1, 29

[[34.5, 30.7, 33.7, 32.7, 32.5, 33.2, 32.9, 34.2, 31.9, 31.3], [31.6, 27.3, 30.8, 29.7, 29.4, 30.3, 30.0, 31.3, 28.6, 28.4], [37.4, 34.1, 36.5, 35.6, 35.5, 36.1, 35.8, 37.1, 35.3, 34.1]]
Discount factor: 0.99, number of games: 60
[[33.8, 32.9, 30.1, 35.0, 35.6, 33.6, 30.2, 33.7, 30.2, 32.5], [30.8, 30.0, 27.8, 31.9, 32.4, 31.2, 27.7, 30.4, 27.4, 29.6], [36.7, 35.8, 32.5, 38.1, 38.8, 36.1, 32.8, 37.0, 32.9, 35.5]]
Discount factor: 0.99, number of games: 70
[[32.9, 33.2, 34.5, 34.7, 29.4, 33.5, 31.0, 33.5, 33.7, 32.0], [29.5, 30.8, 31.1, 32.4, 27.3, 30.7, 27.8, 31.1, 31.0, 29.1], [36.4, 35.7, 37.9, 37.0, 31.5, 36.2, 34.2, 35.9, 36.4, 34.9]]
Discount factor: 0.99, number of games: 80
[[29.3, 32.3, 32.0, 34.9, 33.8, 33.0, 34.7, 32.5, 33.3, 32.6], [26.2, 30.1, 29.7, 32.8, 31.7, 31.3, 32.6, 30.3, 31.0, 30.4], [32.5, 34.5, 34.2, 37.0, 35.9, 34.6, 36.8, 34.7, 35.7, 34.8]]
Discount factor: 0.99, number of games: 90
[[34.1, 31.8, 33.8, 31.8, 32.9, 34.8, 32.5, 32.7, 31.2, 33.6], [32.1, 29.1, 31.1

In [38]:
for discount in discounts:
    total_trials_per_dis = []
    print('Discount factor: ' + str(discount))
    for num_games in games_min:
        trial_run = run(pos_players, all_data, categories, glob_spots, 10, num_games, 30, discount)
        print(trial_run)
        print('Number of games: ' + str(num_games))
        total_trials_per_dis.append(trial_run)
    avg_score_per_discount = np.mean(total_trials_per_dis, axis=0)
    rounded_scores = [round(num, 1) for num in avg_score_per_discount]
    print ('Average score for discount: ' + str(rounded_scores))

Discount factor: 0.5


KeyboardInterrupt: 